In [ ]:
# Wizualizacja Danych, Lab 6 - Jan Banot
# Zadanie 1: Wczytanie przykładowych danych geograficznych i społeczno-ekonomicznych
import pandas as pd
import json
from urllib.request import urlopen

# Wczytanie danych GeoJSON z danymi hrabstw USA
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

# Wczytanie danych CSV o bezrobociu w hrabstwach USA
url_csv = "https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv"
df_unemp = pd.read_csv(url_csv, dtype={"fips": str})

# Wyświetlenie pierwszych wierszy DataFrame
print("Dane o bezrobociu:")
print(df_unemp.head())
print(f"\nLiczba wierszy: {len(df_unemp)}")
print(f"Kolumny: {df_unemp.columns.tolist()}")

# Informacje o danych GeoJSON
print(f"\nLiczba hrabstw w GeoJSON: {len(counties['features'])}")

In [ ]:
# Zadanie 2: Wizualizacja choropleth mapy z Plotly
import plotly.express as px

# Tworzenie interaktywnej mapy choropleth
fig = px.choropleth_mapbox(
    df_unemp,
    geojson=counties,
    locations='fips',
    color='unemp',
    color_continuous_scale="Viridis",
    range_color=(0, 12),
    mapbox_style="carto-positron",
    zoom=3, center={"lat": 37.0902, "lon": -95.7129},
    opacity=0.5,
    labels={'unemp':'Stopa bezrobocia'}
)

# Ustawienie minimalnych marginesów
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# Zadanie 3: Wizualizacja z Plotly – dane światowe
import plotly.express as px

# Załadowanie danych Gapminder dla roku 2007
df = px.data.gapminder().query("year==2007")

# Wyświetlenie podstawowych informacji o danych
print("Dane Gapminder (rok 2007):")
print(df.head())
print(f"\nLiczba krajów: {len(df)}")
print(f"Kontynenty: {df['continent'].unique().tolist()}")

In [ ]:
# Tworzenie interaktywnej mapy z punktami reprezentującymi kraje
fig = px.scatter_geo(
    df,
    locations="iso_alpha",
    color="continent",
    hover_name="country",
    size="pop",
    projection="natural earth",
    title="Populacja krajów świata w 2007 roku"
)

fig.show()

In [ ]:
# Zadanie 4: Wizualizacja z ipyleaflet – mapy interaktywne
from ipyleaflet import Map, Marker, Popup
from ipywidgets import HTML

# Utwórz mapę z domyślnym środkiem (centrum Polski) i zoomem
mapa = Map(center=(52, 19), zoom=6)

# Lista punktów - stolice wybranych województw
locations = [
    {"name": "Warszawa", "lat": 52.2297, "lon": 21.0118},
    {"name": "Wrocław", "lat": 51.1079, "lon": 17.0385},
    {"name": "Kraków", "lat": 50.0647, "lon": 19.9450}
]

# Dodaj markery
for loc in locations:
    marker = Marker(location=(loc["lat"], loc["lon"]))
    marker.popup = Popup(
        child=HTML(f"<b>{loc['name']}</b>"),
        close_button=False,
        auto_close=False,
        close_on_click=False
    )
    mapa.add_layer(marker)

mapa


In [ ]:
# Zadanie 5: Tworzenie interaktywnej mapy z ipyleaflet – część 2
from ipyleaflet import Map, Marker, Popup
from ipywidgets import HTML, Output
from IPython.display import display

# Utwórz mapę z domyślnym środkiem (centrum Polski) i zoomem
mapa2 = Map(center=(52, 19), zoom=6)

# Lista punktów z dodatkowymi danymi - stolice wybranych województw
locations_extended = [
    {
        "name": "Warszawa",
        "lat": 52.2297,
        "lon": 21.0118,
        "population": 1_793_579,
        "area_km2": 517.24,
        "voivodeship": "mazowieckie",
    },
    {
        "name": "Wrocław",
        "lat": 51.1079,
        "lon": 17.0385,
        "population": 674_079,
        "area_km2": 292.82,
        "voivodeship": "dolnośląskie",
    },
    {
        "name": "Kraków",
        "lat": 50.0647,
        "lon": 19.9450,
        "population": 804_237,
        "area_km2": 326.85,
        "voivodeship": "małopolskie",
    },
]

# Output widget do wyświetlania informacji po kliknięciu
output = Output()


def create_click_handler(loc_data):
    """Tworzy funkcję obsługującą kliknięcie dla danego markera"""

    def on_click(**kwargs):
        with output:
            output.clear_output()
            print(f"=== {loc_data['name']} ===")
            print(f"Województwo: {loc_data['voivodeship']}")
            print(f"Populacja: {loc_data['population']:,}".replace(",", " "))
            print(f"Powierzchnia: {loc_data['area_km2']} km²")
            print(
                f"Gęstość: {loc_data['population'] / loc_data['area_km2']:.0f} os./km²"
            )
            print(f"Współrzędne: {loc_data['lat']}°N, {loc_data['lon']}°E")

    return on_click


# Dodaj markery z rozszerzonymi danymi
for loc in locations_extended:
    # Tworzenie popupu z podstawowymi informacjami
    popup_content = HTML(
        f"""
        <b>{loc["name"]}</b><br>
        Województwo: {loc["voivodeship"]}<br>
        Populacja: {loc["population"]:,}<br>
        Powierzchnia: {loc["area_km2"]} km²
    """.replace(",", " ")
    )

    marker = Marker(location=(loc["lat"], loc["lon"]))
    marker.popup = Popup(
        child=popup_content, close_button=True, auto_close=True, close_on_click=False
    )

    # Dodanie obsługi zdarzenia on_click
    marker.on_click(create_click_handler(loc))

    mapa2.add_layer(marker)

# Wyświetl mapę i output
display(mapa2)
display(output)

Zadanie 6: Porównanie Plotly i ipyleaflet – zadanie projektowe
# Raport: Porównanie bibliotek Plotly i ipyleaflet

## 1. Łatwość użycia

| Aspekt                      | Plotly                                      | ipyleaflet                                   |
| --------------------------- | ------------------------------------------- | -------------------------------------------- |
| **Krzywa uczenia**          | Niska - intuicyjne API z `plotly.express`   | Średnia - wymaga znajomości widgetów Jupyter |
| **Dokumentacja**            | Obszerna, wiele przykładów                  | Dobra, ale mniej rozbudowana                 |
| **Szybkość prototypowania** | Bardzo szybka - jedna linijka kodu dla mapy | Wymaga więcej kodu do podstawowej mapy       |

**Plotly** wyróżnia się prostotą - funkcje jak `px.choropleth()` czy `px.scatter_geo()` pozwalają stworzyć mapę w kilku linijkach.
**ipyleaflet** wymaga ręcznego dodawania warstw i markerów, ale daje większą kontrolę.

---

## 2. Możliwości dekoracji i personalizacji

| Funkcja                    | Plotly                            | ipyleaflet                                    |
| -------------------------- | --------------------------------- | --------------------------------------------- |
| **Style map bazowych**     | Kilka wbudowanych (carto, stamen) | Wiele dostawców (OpenStreetMap, Stamen, ESRI) |
| **Niestandardowe markery** | Ograniczone                       | Pełna kontrola (ikony, kształty)              |
| **Warstwy GeoJSON**        | Tak                               | Tak                                           |
| **Skale kolorów**          | Bogate wbudowane palety           | Wymaga ręcznej konfiguracji                   |
| **Animacje**               | Tak (slider czasowy)              | Ograniczone                                   |

**Plotly** lepiej nadaje się do map statystycznych (choropleth).
**ipyleaflet** oferuje większą elastyczność w dostosowywaniu wyglądu markerów i warstw.

---

## 3. Interaktywność

| Funkcja                     | Plotly                 | ipyleaflet                   |
| --------------------------- | ---------------------- | ---------------------------- |
| **Zoom/Pan**                | ✅                     | ✅                           |
| **Hover tooltips**          | ✅ Wbudowane           | ⚠️ Wymaga konfiguracji       |
| **Obsługa zdarzeń (click)** | ⚠️ Ograniczona         | ✅ Pełna (on_click, on_move) |
| **Dynamiczna aktualizacja** | Przez callbacks (Dash) | Natywna przez widgety        |
| **Rysowanie na mapie**      | ❌                     | ✅ (DrawControl)             |

**ipyleaflet** oferuje lepszą obsługę zdarzeń użytkownika i możliwość rysowania.  
**Plotly** ma lepsze wbudowane tooltips i animacje.

---

## 4. Wydajność i obsługiwane środowiska

| Aspekt                 | Plotly              | ipyleaflet                     |
| ---------------------- | ------------------- | ------------------------------ |
| **Jupyter Notebook**   | ✅                  | ✅                             |
| **JupyterLab**         | ✅                  | ✅                             |
| **Eksport do HTML**    | ✅ Łatwy            | ⚠️ Wymaga dodatkowych narzędzi |
| **Dashboardy**         | ✅ Dash             | ✅ Voilà                       |
| **Duże zbiory danych** | Dobra (WebGL)       | Średnia                        |
| **Renderowanie**       | Klient (JavaScript) | Klient (JavaScript)            |

**Plotly** lepiej radzi sobie z dużymi zbiorami danych dzięki WebGL.
**ipyleaflet** jest lżejszy, ale może spowolnić przy wielu markerach.

---

## Podsumowanie

| Kryterium                        | Lepsza biblioteka |
| -------------------------------- | ----------------- |
| Szybkie prototypowanie           | **Plotly**        |
| Mapy statystyczne (choropleth)   | **Plotly**        |
| Pełna kontrola nad interakcją    | **ipyleaflet**    |
| Niestandardowe markery i warstwy | **ipyleaflet**    |
| Eksport i dashboardy             | **Plotly**        |
| Rysowanie i edycja na mapie      | **ipyleaflet**    |

**Rekomendacja:**

- Użyj **Plotly** do wizualizacji danych geograficznych i statystycznych, jeśli produkt końcowy to raport (statyczny plik HTML, PDF, prezentacja)
- Użyj **ipyleaflet** do aplikacji wymagających pełnej interakcji użytkownika z mapą.


In [ ]:
# Zadanie 7: Rozszerzenie wizualizacji
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Obliczenie statystyk
mean_unemp = df_unemp['unemp'].mean()
median_unemp = df_unemp['unemp'].median()
std_unemp = df_unemp['unemp'].std()
q1 = df_unemp['unemp'].quantile(0.25)
q3 = df_unemp['unemp'].quantile(0.75)

# Tworzenie subplots: mapa u góry, histogram na dole
fig = make_subplots(
    rows=2, cols=1,
    row_heights=[0.65, 0.35],
    specs=[[{"type": "mapbox"}], [{"type": "xy"}]],
    vertical_spacing=0.08,
    subplot_titles=("Mapa bezrobocia w hrabstwach USA", "Rozkład histograficzny bezrobocia")
)

# Dodanie mapy choropleth (górny panel)
choropleth = go.Choroplethmapbox(
    geojson=counties,
    locations=df_unemp['fips'],
    z=df_unemp['unemp'],
    colorscale="Viridis",
    zmin=0,
    zmax=12,
    marker_opacity=0.5,
    marker_line_width=0,
    colorbar=dict(title="Stopa<br>bezrobocia (%)", y=0.75, len=0.45),
    hovertemplate="FIPS: %{location}<br>Bezrobocie: %{z:.1f}%<extra></extra>"
)
fig.add_trace(choropleth, row=1, col=1)

# Dodanie histogramu (dolny panel)
fig.add_trace(
    go.Histogram(
        x=df_unemp['unemp'],
        nbinsx=30,
        name="Rozkład bezrobocia",
        marker_color='#440154',
        opacity=0.7,
        hovertemplate="Bezrobocie: %{x:.1f}%<br>Liczba hrabstw: %{y}<extra></extra>"
    ),
    row=2, col=1
)

# Dodanie linii pionowych dla średniej i mediany na histogramie
fig.add_shape(
    type="line",
    x0=mean_unemp, x1=mean_unemp,
    y0=0, y1=1000,
    line=dict(color="red", width=2, dash="dash"),
    xref="x", yref="y"
)

fig.add_shape(
    type="line",
    x0=median_unemp, x1=median_unemp,
    y0=0, y1=1000,
    line=dict(color="blue", width=2, dash="dot"),
    xref="x", yref="y"
)

# Adnotacje dla średniej i mediany
fig.add_annotation(
    x=mean_unemp + 1.5, y=420,
    text=f"Średnia: {mean_unemp:.1f}%",
    showarrow=False,
    font=dict(color="red", size=11),
    xref="x", yref="y"
)

fig.add_annotation(
    x=median_unemp - 1.5, y=380,
    text=f"Mediana: {median_unemp:.1f}%",
    showarrow=False,
    font=dict(color="blue", size=11),
    xref="x", yref="y"
)

# Adnotacja ze statystykami
stats_text = (
    f"<b>Statystyki:</b><br>"
    f"Średnia: {mean_unemp:.2f}% | Mediana: {median_unemp:.2f}%<br>"
    f"Odch. std.: {std_unemp:.2f}% | Min: {df_unemp['unemp'].min():.1f}% | Max: {df_unemp['unemp'].max():.1f}%<br>"
    f"Q1: {q1:.1f}% | Q3: {q3:.1f}% | IQR: {q3-q1:.1f}%"
)

fig.add_annotation(
    x=18, y=350,
    text=stats_text,
    showarrow=False,
    font=dict(size=10),
    align="left",
    bgcolor="rgba(255,255,255,0.8)",
    bordercolor="gray",
    borderwidth=1,
    xref="x", yref="y"
)

# Konfiguracja layoutu
fig.update_layout(
    title=dict(
        text="Bezrobocie w hrabstwach USA - Mapa i analiza histograficzna",
        x=0.5,
        font=dict(size=18)
    ),
    mapbox=dict(
        style="carto-positron",
        zoom=3,
        center={"lat": 37.0902, "lon": -95.7129}
    ),
    margin={"r": 20, "t": 80, "l": 60, "b": 60},
    showlegend=False,
    height=900
)

# Konfiguracja osi histogramu
fig.update_xaxes(title_text="Stopa bezrobocia (%)", row=2, col=1)
fig.update_yaxes(title_text="Liczba hrabstw", row=2, col=1)

fig.show()

In [ ]:
# Zadanie 8: Stacje wodociągowe
from ipyleaflet import Map, Marker, Popup, AwesomeIcon, WidgetControl, LayerGroup
from ipywidgets import HTML, Output, Dropdown
from IPython.display import display

# Dane stacji wodociągowych
stacje_wodociagowe = [
    {
        "name": "Stacja A",
        "lat": 52.2297,
        "lon": 21.0122,
        "year_built": 1980,
        "last_inspection": "2024-03-15",
        "status": "Dobry",
        "connected_residents": 15000,
        "city": "Warszawa",
    },
    {
        "name": "Stacja B",
        "lat": 50.0615,
        "lon": 19.9362,
        "year_built": 1975,
        "last_inspection": "2023-11-10",
        "status": "Zły",
        "connected_residents": 8000,
        "city": "Kraków",
    },
    {
        "name": "Stacja C",
        "lat": 51.1079,
        "lon": 17.0385,
        "year_built": 1990,
        "last_inspection": "2024-01-20",
        "status": "Wymaga remontu",
        "connected_residents": 12000,
        "city": "Wrocław",
    },
]

# Mapowanie statusu na kolor
status_colors = {"Dobry": "green", "Zły": "red", "Wymaga remontu": "orange"}

status_icons = {
    "Dobry": "tint",
    "Zły": "exclamation-triangle",
    "Wymaga remontu": "wrench",
}

# Utwórz mapę Polski
mapa_stacje = Map(center=(51.9194, 19.1451), zoom=6)

# Output widget do wyświetlania szczegółowych informacji
output_stacje = Output()

# Słownik do przechowywania markerów według statusu
markers_by_status = {"Dobry": [], "Zły": [], "Wymaga remontu": []}

# LayerGroup dla markerów
all_markers_layer = LayerGroup()


def create_marker_click_handler(stacja):
    """Tworzy funkcję obsługującą kliknięcie dla danego markera"""

    def on_click(**kwargs):
        with output_stacje:
            output_stacje.clear_output()
            print(f"{'=' * 40}")
            print(f"🏭 {stacja['name']} ({stacja['city']})")
            print(f"{'=' * 40}")
            print(f"📅 Rok budowy: {stacja['year_built']}")
            print(f"🔍 Ostatnia kontrola: {stacja['last_inspection']}")
            print(f"📊 Stan techniczny: {stacja['status']}")
            print(
                f"👥 Podłączonych mieszkańców: {stacja['connected_residents']:,}".replace(
                    ",", " "
                )
            )
            print(f"📍 Współrzędne: {stacja['lat']}°N, {stacja['lon']}°E")

            # Dodatkowe informacje
            wiek = 2024 - stacja["year_built"]
            print(f"\n📈 Wiek stacji: {wiek} lat")

    return on_click


# Tworzenie markerów dla każdej stacji
for stacja in stacje_wodociagowe:
    # Tworzenie ikony z odpowiednim kolorem
    icon = AwesomeIcon(
        name=status_icons[stacja["status"]],
        marker_color=status_colors[stacja["status"]],
        icon_color="white",
        spin=False,
    )

    # Tworzenie popupu z informacjami
    popup_html = f"""
    <div style="min-width: 200px;">
        <h4 style="margin: 0 0 10px 0; color: {status_colors[stacja["status"]]};">
            🏭 {stacja["name"]}
        </h4>
        <p style="margin: 5px 0;"><b>Miasto:</b> {stacja["city"]}</p>
        <p style="margin: 5px 0;"><b>Rok budowy:</b> {stacja["year_built"]}</p>
        <p style="margin: 5px 0;"><b>Ostatnia kontrola:</b> {stacja["last_inspection"]}</p>
        <p style="margin: 5px 0;"><b>Stan:</b> 
            <span style="color: {status_colors[stacja["status"]]}; font-weight: bold;">
                {stacja["status"]}
            </span>
        </p>
        <p style="margin: 5px 0;"><b>Mieszkańców:</b> {stacja["connected_residents"]:,}</p>
    </div>
    """.replace(",", " ")

    marker = Marker(location=(stacja["lat"], stacja["lon"]), icon=icon, draggable=False)

    marker.popup = Popup(
        child=HTML(popup_html),
        close_button=True,
        auto_close=True,
        close_on_click=False,
        max_width=300,
    )

    # Dodanie obsługi kliknięcia
    marker.on_click(create_marker_click_handler(stacja))

    # Zapisz marker do odpowiedniej kategorii
    markers_by_status[stacja["status"]].append(marker)
    all_markers_layer.add(marker)

# Dodaj warstwę z markerami do mapy
mapa_stacje.add(all_markers_layer)

# Dropdown do filtrowania
filter_dropdown = Dropdown(
    options=["Wszystkie", "Dobry", "Zły", "Wymaga remontu"],
    value="Wszystkie",
    description="Filtruj:",
    style={"description_width": "initial"},
)


def filter_markers(change):
    """Funkcja filtrująca markery według stanu technicznego"""
    selected = change["new"]

    # Usuń wszystkie markery z warstwy
    all_markers_layer.clear_layers()

    # Dodaj odpowiednie markery
    if selected == "Wszystkie":
        for status_markers in markers_by_status.values():
            for marker in status_markers:
                all_markers_layer.add(marker)
    else:
        for marker in markers_by_status[selected]:
            all_markers_layer.add(marker)

    # Aktualizuj output
    with output_stacje:
        output_stacje.clear_output()
        if selected == "Wszystkie":
            count = sum(len(m) for m in markers_by_status.values())
            print(f"Wyświetlono wszystkie stacje ({count})")
        else:
            count = len(markers_by_status[selected])
            print(f"Wyświetlono stacje o stanie: {selected} ({count})")


filter_dropdown.observe(filter_markers, names="value")

# Dodaj kontrolkę filtrowania do mapy
filter_control = WidgetControl(widget=filter_dropdown, position="topright")
mapa_stacje.add(filter_control)

# Legenda
legend_html = HTML("""
<div style="padding: 10px; background: white; border-radius: 5px; border: 1px solid gray;">
    <b>Legenda - Stan techniczny:</b><br>
    <span style="color: green;">● Dobry</span><br>
    <span style="color: orange;">● Wymaga remontu</span><br>
    <span style="color: red;">● Zły</span>
</div>
""")
legend_control = WidgetControl(widget=legend_html, position="bottomright")
mapa_stacje.add(legend_control)

# Wyświetlenie mapy i panelu informacyjnego
print("🗺️ Mapa stacji wodociągowych w Polsce")
print("Kliknij na marker, aby zobaczyć szczegółowe informacje.")
print("-" * 50)
display(mapa_stacje)
display(output_stacje)